In [1]:
%pip install mediapipe opencv-python pandas scikit-learn
%pip install --upgrade tensorflow tensorflow-addons


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow info messages

In [3]:
import mediapipe as mp 
import cv2 

In [4]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [5]:
# Open the camera (you can change the index as needed)
cap = cv2.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Break the loop if there's an issue with the camera
        if not ret:
            print("Error: Could not read frame.")
            break

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw landmarks on the image
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                  mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                                  )

        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                                  )

        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                                  )

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        cv2.imshow('Holistic Model', image)

        # Exit the program when 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

2023-11-20 11:08:55.090 Python[1834:38674] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1700464136.525469       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2023-11-20 11:08:56.732 Python[1834:38674] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [6]:
import csv
import numpy as np
import os

In [7]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)

In [8]:
num_coords

501

In [9]:
landmarks = ['class']
for val in range(1,num_coords+1):
    landmarks += ['x{}'.format(val) ,'y{}'.format(val),'z{}'.format(val),'v{}'.format(val) ]

In [10]:
with open('coords.csv' , mode='w',newline='') as f:
    csv_writer = csv.writer(f , delimiter=',',quotechar='"',quoting = csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [15]:
class_name = "Surprised"

In [16]:
cap = cv2.VideoCapture(0)

# initialize the model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        ret , frame = cap.read()

        image = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)

        image.flags.writeable = False

        results = holistic.process(image)

        # print(results.pose_landmarks)
        image.flags.writeable = True

        image = cv2.cvtColor(image , cv2.COLOR_RGB2BGR)

        # face
        mp_drawing.draw_landmarks(image , results.face_landmarks , mp_holistic.FACEMESH_TESSELATION,
        mp_drawing.DrawingSpec(color=(80,110,10) ,thickness = 1 ,circle_radius=1 ) ,         mp_drawing.DrawingSpec(color=(80,256,121) ,thickness = 1 ,circle_radius=1 ))

        # right hand
        mp_drawing.draw_landmarks(image , results.right_hand_landmarks , mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,22,10) ,thickness = 2 ,circle_radius=4 ) ,          mp_drawing.DrawingSpec(color=(80,44,121) ,thickness = 2 ,circle_radius=2 ) )
        
        # left hand
        mp_drawing.draw_landmarks(image , results.left_hand_landmarks , mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121,22,76) ,thickness = 2 ,circle_radius=4 ) ,         mp_drawing.DrawingSpec(color=(121,44,230) ,thickness = 2 ,circle_radius=2 ))
        
        # pose detection
        mp_drawing.draw_landmarks(image , results.pose_landmarks , mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245,117,66) ,thickness = 2 ,circle_radius=4 ) ,        mp_drawing.DrawingSpec(color=(245,66,230) ,thickness = 2 ,circle_radius=2 ))

        # export coordinates
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x , landmark.y , landmark.z , landmark.visibility] for landmark in pose]).flatten())

            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x , landmark.y , landmark.z , landmark.visibility] for landmark in face]).flatten())

            row = pose_row + face_row
            row.insert(0 , class_name)

            with open('coords.csv' , mode='a',newline='') as f:
                csv_writer = csv.writer(f , delimiter=',',quotechar='"',quoting = csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass
        
        cv2.imshow("Holistic Model Detections",image)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1700464311.658819       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1 Pro


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [18]:
df = pd.read_csv('coords.csv')

In [19]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Sad,0.522746,0.622452,-0.989480,0.999040,0.545296,0.553469,-0.931817,0.997784,0.561573,...,0.004496,0.0,0.580673,0.557727,0.027024,0.0,0.585466,0.550592,0.028057,0.0
1,Sad,0.519844,0.625996,-1.120764,0.998870,0.544039,0.554894,-1.065956,0.997689,0.560592,...,0.005339,0.0,0.581250,0.555694,0.027122,0.0,0.586041,0.548549,0.028057,0.0
2,Sad,0.518664,0.630015,-1.126691,0.998695,0.543627,0.556782,-1.071064,0.997621,0.560216,...,0.004885,0.0,0.582455,0.556380,0.026961,0.0,0.587284,0.548885,0.027922,0.0
3,Sad,0.517678,0.632889,-1.116301,0.998476,0.543175,0.558471,-1.062319,0.997510,0.559802,...,0.004257,0.0,0.582405,0.555869,0.026180,0.0,0.587201,0.548425,0.027113,0.0
4,Sad,0.517467,0.634157,-1.166500,0.998260,0.543085,0.559137,-1.110740,0.997315,0.559679,...,0.004428,0.0,0.582393,0.555528,0.026624,0.0,0.587181,0.548461,0.027562,0.0


In [20]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
664,Surprised,0.511635,0.689084,-1.063425,0.998061,0.539984,0.599289,-1.012165,0.996202,0.556125,...,-0.008533,0.0,0.585869,0.584677,0.014849,0.0,0.591029,0.576792,0.015642,0.0
665,Surprised,0.509511,0.690259,-1.103062,0.998226,0.538321,0.600661,-1.055266,0.996521,0.554942,...,-0.008439,0.0,0.585399,0.585042,0.015015,0.0,0.590603,0.577241,0.015807,0.0
666,Surprised,0.508409,0.691063,-1.099262,0.998381,0.537460,0.601792,-1.050094,0.996813,0.554320,...,-0.008461,0.0,0.584729,0.585676,0.014971,0.0,0.589952,0.577740,0.015786,0.0
667,Surprised,0.507590,0.691912,-1.175115,0.998525,0.536940,0.602682,-1.126617,0.997092,0.553979,...,-0.008585,0.0,0.584903,0.585753,0.014688,0.0,0.590089,0.577773,0.015474,0.0
668,Surprised,0.506377,0.692860,-1.140855,0.998646,0.536132,0.603888,-1.104056,0.997319,0.553536,...,-0.009174,0.0,0.585201,0.586320,0.013791,0.0,0.590404,0.578502,0.014538,0.0


In [21]:
x = df.iloc[: , 1:]
y = df.iloc[: , 0]

In [22]:
X_train , X_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 , random_state=0)

In [23]:
len(X_train) , len(X_test)

(535, 134)

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression , RidgeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

In [25]:
pipelines = {
    'lr':make_pipeline(StandardScaler() , LogisticRegression()),
    'rc':make_pipeline(StandardScaler() , RidgeClassifier()),
    'rf':make_pipeline(StandardScaler() , RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler() , GradientBoostingClassifier())
}

In [26]:
list(pipelines.values())[0]

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [27]:
pipeline = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))
model = pipeline.fit(X_train, y_train)

In [28]:
fit_models = {}
for algo, pipeline in pipelines.items():
    fit_models[algo] = model

In [29]:
fit_models['rc'].predict(X_test)

array(['Surprised', 'Happy', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy',
       'Happy', 'Surprised', 'Happy', 'Sad', 'Happy', 'Happy',
       'Surprised', 'Surprised', 'Sad', 'Happy', 'Surprised', 'Sad',
       'Surprised', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy',
       'Sad', 'Happy', 'Happy', 'Happy', 'Happy', 'Surprised', 'Happy',
       'Sad', 'Happy', 'Sad', 'Sad', 'Happy', 'Happy', 'Happy', 'Sad',
       'Happy', 'Sad', 'Sad', 'Sad', 'Happy', 'Surprised', 'Surprised',
       'Sad', 'Happy', 'Surprised', 'Happy', 'Sad', 'Happy', 'Sad',
       'Happy', 'Sad', 'Surprised', 'Surprised', 'Sad', 'Happy', 'Happy',
       'Happy', 'Sad', 'Surprised', 'Happy', 'Happy', 'Happy',
       'Surprised', 'Sad', 'Sad', 'Happy', 'Happy', 'Sad', 'Sad', 'Sad',
       'Happy', 'Sad', 'Happy', 'Happy', 'Happy', 'Surprised', 'Happy',
       'Sad', 'Happy', 'Happy', 'Surprised', 'Happy', 'Sad', 'Surprised',
       'Happy', 'Sad', 'Sad', 'Sad', 'Sad', 'Surprised', 'Happy', 'Happy',
       'Sad'

In [30]:
fit_models['lr'].predict(X_test)

array(['Surprised', 'Happy', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy',
       'Happy', 'Surprised', 'Happy', 'Sad', 'Happy', 'Happy',
       'Surprised', 'Surprised', 'Sad', 'Happy', 'Surprised', 'Sad',
       'Surprised', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy',
       'Sad', 'Happy', 'Happy', 'Happy', 'Happy', 'Surprised', 'Happy',
       'Sad', 'Happy', 'Sad', 'Sad', 'Happy', 'Happy', 'Happy', 'Sad',
       'Happy', 'Sad', 'Sad', 'Sad', 'Happy', 'Surprised', 'Surprised',
       'Sad', 'Happy', 'Surprised', 'Happy', 'Sad', 'Happy', 'Sad',
       'Happy', 'Sad', 'Surprised', 'Surprised', 'Sad', 'Happy', 'Happy',
       'Happy', 'Sad', 'Surprised', 'Happy', 'Happy', 'Happy',
       'Surprised', 'Sad', 'Sad', 'Happy', 'Happy', 'Sad', 'Sad', 'Sad',
       'Happy', 'Sad', 'Happy', 'Happy', 'Happy', 'Surprised', 'Happy',
       'Sad', 'Happy', 'Happy', 'Surprised', 'Happy', 'Sad', 'Surprised',
       'Happy', 'Sad', 'Sad', 'Sad', 'Sad', 'Surprised', 'Happy', 'Happy',
       'Sad'

In [31]:
from sklearn.metrics import accuracy_score
import pickle

In [32]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [33]:
fit_models['lr'].predict(X_test)

array(['Surprised', 'Happy', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy',
       'Happy', 'Surprised', 'Happy', 'Sad', 'Happy', 'Happy',
       'Surprised', 'Surprised', 'Sad', 'Happy', 'Surprised', 'Sad',
       'Surprised', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy',
       'Sad', 'Happy', 'Happy', 'Happy', 'Happy', 'Surprised', 'Happy',
       'Sad', 'Happy', 'Sad', 'Sad', 'Happy', 'Happy', 'Happy', 'Sad',
       'Happy', 'Sad', 'Sad', 'Sad', 'Happy', 'Surprised', 'Surprised',
       'Sad', 'Happy', 'Surprised', 'Happy', 'Sad', 'Happy', 'Sad',
       'Happy', 'Sad', 'Surprised', 'Surprised', 'Sad', 'Happy', 'Happy',
       'Happy', 'Sad', 'Surprised', 'Happy', 'Happy', 'Happy',
       'Surprised', 'Sad', 'Sad', 'Happy', 'Happy', 'Sad', 'Sad', 'Sad',
       'Happy', 'Sad', 'Happy', 'Happy', 'Happy', 'Surprised', 'Happy',
       'Sad', 'Happy', 'Happy', 'Surprised', 'Happy', 'Sad', 'Surprised',
       'Happy', 'Sad', 'Sad', 'Sad', 'Sad', 'Surprised', 'Happy', 'Happy',
       'Sad'

In [34]:
y_test

580    Surprised
310        Happy
14           Sad
542        Happy
76           Sad
         ...    
489        Happy
557    Surprised
187          Sad
485        Happy
101          Sad
Name: class, Length: 134, dtype: object

In [35]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)

In [36]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [37]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(max_iter=1000))])

In [38]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1700464383.016289       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1 Pro
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Framework

Sad [7.70956814e-25 1.00000000e+00 2.19414817e-10]
Sad [2.71495308e-21 9.99999996e-01 3.93580582e-09]
Sad [8.22024944e-19 9.99999994e-01 6.02400569e-09]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Sad [1.57911917e-15 9.99999898e-01 1.02275583e-07]
Sad [1.84599693e-14 9.99998280e-01 1.71955610e-06]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have va

...

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have va

Happy [9.99415411e-01 1.24673348e-06 5.83342111e-04]
Happy [0.97376376 0.00133647 0.02489977]
Happy [0.96410497 0.00112747 0.03476755]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Happy [0.94016627 0.00108516 0.05874857]
Happy [7.60230852e-01 1.52853505e-04 2.39616295e-01]
Happy [6.68994211e-01 1.93858878e-05 3.30986403e-01]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [39]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))

(405, 289)